##### Now that we have the relevant context, lets decide on priorities for SUs in their individual locations:
```
def priority_engine():
    target_bands = [(12.2, 12.7), .... , ()]
    for location in locations:
        users = get_users_in_current_location()
        
        # Relevant for all SUs in a region or diff SUs may have diff scores based on their LOCATION in a REGION
        user_distribution_priority = get_SU_distribution_priority()
        pu_distribution_priority = get_PU_distribution_priority()
        
        
        
        for user in users:
            # Check user constraints
            if user.op_band in target_bands:
                # Calculate priority based on which we might need to move the user into the target band
            else:
                # As constraints are violated, user is dropped from consideration

            # Goal is to generate a score
            class_priority_score = get_class_priority(user['class'])
            location_priority_score = get_location_priority(user['location'])
            mobility_priority_score = get_mobility_priority(user['mobile'])
            traffic_priority_score = get_traffic_priority(user['traffic_type'])

            score = sum([all_priorities])
```
##### To consider:
* Divide overall space into multiple regions and every region into multiple locations.
    * Every region may have its own SU/PU distribution, available spectrum, policies, etc.
    * Operational context may vary from one location to another.
* Simulate channel characteristics - number & distribution of SU/PU, fading, path loss, etc. 

In [1]:
import pandas as pd
import numpy as np
import json
import json
import random
import os

pd.set_option("display.max_rows", None, "display.max_columns", None,'display.max_colwidth', None)

Create Data files

In [2]:
def get_user_classes():
    return ['ordinary', 'emergency', 'scientific', 'government']


def get_class_priorities():
    return {
        'ordinary': 1,
        'scientific': 2,
        'government': 3,
        'emergency': 4
    }


def get_locations():
    return [f"loc{i + 1}" for i in range(30)]


def get_fading_levels():
    return ['low', 'average', 'high', 'severe']


def get_fading_priority():
    return {
        'low': 1,
        'average': 2,
        'high': 3,
        'severe': 4
    }


def get_bands():
    return {
        "n1": [1920, 1980],
        "n2": [1850, 1910],
        "n3": [1710, 1785],
        "n5": [824, 849],
        "n7": [2500, 2570],
        "n30": [2305, 2315],
        "n34": [2010, 2025],
        "n38": [2570, 2620],
        "n40": [2300, 2400],
        "n41": [2496, 2690],
        "n48": [3550, 3700],
        "n78": [3300, 3800],
        "n79": [4400, 5000]
    }


def get_operating_bands(user_class):
    all_bands = [(key, value) for key, value in get_bands().items()]
    
    if user_class in ['ordinary', 'scientific']:
        number_of_bands = random.randint(1, 2)
    elif user_class in ['government', 'emergency']:
        number_of_bands = random.randint(2, 5)
        
    sampled_list = random.sample(all_bands, number_of_bands)
    
    current_band = random.choice(sampled_list)[1]
    op_freq = random.randint(current_band[0], current_band[1])
    op_bands = [item[1] for item in sampled_list]
    
    return op_freq, op_bands


def get_user_data():
    columns = ['user_id', 'location', 'user_type', 'freq_min', 'freq_max']
    mobile = [True, False]
    user_type = get_user_classes()
    locations = get_locations()
    data = []

    for i in range(500):
        user_class = random.choice(user_type)
        current_band, op_bands = get_operating_bands(user_class)
        data.append({
            'id': str(i + 1),
            'location': random.choice(locations),
            'class': user_class,
            'op_freq': current_band,
            'allowed_bands': op_bands
        })
    
    return data

def get_location_data():
    columns = ['location', 'fading']
    weather = get_fading_levels()
    locations = get_locations()
    data = []

    for i in range(len(locations)):
        data.append({
            'location': locations[i],
            'fading': random.choice(weather)
        })

    return data

Import data files

In [3]:
location_data = pd.DataFrame(get_location_data())
location_data.head(5)

,location,fading
0,loc1,average
1,loc2,low
2,loc3,average
3,loc4,low
4,loc5,high


In [4]:
user_data = pd.DataFrame(get_user_data())
user_data.head(5)

,id,location,class,op_freq,allowed_bands
0,1,loc7,government,3666,"[[2305, 2315], [1850, 1910], [2500, 2570], [1920, 1980], [3550, 3700]]"
1,2,loc2,ordinary,2021,"[[2010, 2025]]"
2,3,loc11,scientific,2311,"[[2305, 2315]]"
3,4,loc18,emergency,1719,"[[2010, 2025], [1710, 1785], [2500, 2570]]"
4,5,loc20,government,2579,"[[1850, 1910], [2570, 2620], [3550, 3700]]"


Get score based on user's class/heirarchy

In [5]:
def get_class_priority(user_class):
    priorities = get_class_priorities()
    
    return priorities.get(user_class, 0)

Get score based on user's location. Score may be affected by multiple factors

In [6]:
def get_location_priority(user_location: str):
    # fading based priority
    fading_level = location_data.loc[location_data['location'] == user_location, 'fading'].item()
    priorities_based_on_fading = get_fading_priority()
    # Other criterias are also possible
    
    return sum([
        priorities_based_on_fading.get(fading_level, 0)
    ])

Get all users in a particular location

In [7]:
def get_users_in_location(location):
    returnable = []
    users = user_data.loc[user_data['location'] == location, 'id']
    
    return list(users)

Check if user's operational constraints are satisfied

In [8]:
def operating_freq_constraint(target_band, user_id):    
    allowed_bands = list(user_data.loc[user_data['id'] == user_id, 'allowed_bands'].values[0])
    
    #     [10, 20] -> [9, 15], [15, 21], [5, 25]
    if PARTIAL_BAND_SUPPORT:
        for band in allowed_bands:
            min_allowed_freq, max_allowed_freq = band[0], band[1]
            min_target_freq, max_target_freq = target_band[0], target_band[1]

            if min_allowed_freq <= min_target_freq and max_allowed_freq >= max_target_freq:
                return True, {}
            elif min_allowed_freq <= min_target_freq and max_allowed_freq <= max_target_freq:
                return True, {}
            elif min_allowed_freq >= min_target_freq and max_allowed_freq >= max_target_freq:
                return True, {}
    else:
        if target_band in allowed_bands:
            return True, {}
            
        
    return False, {
        'target_bands': [target_band],
        'allowed': allowed_bands
    }

In [9]:
def use_rule_based_algorithm(locations, target_bands):
    priority_dict = dict()
    invalid_dict = dict()
    
    for location in locations:
        priority_dict[location] = None
        users = get_users_in_location(location)
        
        location_dict = {}
        for target_band in target_bands:
            band_key = str(target_band[0]) + "-" + str(target_band[1])
            band_data = {}
            for user in users:
                class_score, location_score, constraint_score = 0, 0, 0.0
                
                # Check user constraints
                constraint_flag, data = operating_freq_constraint(target_band, user)
                if not constraint_flag:
                    if user not in invalid_dict:
                        data['loc'] = location
                        invalid_dict[user] = data
                    else:
                        invalid_dict[user]['target_bands'].extend(data['target_bands'])
                    constraint_score = float('-inf')
                
                # class based rank
                class_score = get_class_priority(user_data.loc[user_data['id'] == user, 'class'].item())
                
                # location based rank
                location_score = get_location_priority(location)
                
                total_score = sum([class_score, constraint_score, location_score])
                
                if total_score > 0:
                    band_data[user] = total_score
            
            location_dict[band_key] = band_data
        
        priority_dict[location] = location_dict
    
    return priority_dict, invalid_dict

In [10]:
def priority_engine(algorithm, target_bands: list):
    locations_under_consideration = get_locations()
    
    if algorithm == 'RULE_BASED':
        priorities = use_rule_based_algorithm(locations_under_consideration, target_bands)
    elif algorithm == 'MACHINE_BASED':
        priorities = None
    
    return priorities

# test
ALGORITHM = 'RULE_BASED'
PARTIAL_BAND_SUPPORT = False
target_bands = [[3550, 3700], [2010, 2025]]

calculated_priorities, invalid_cases = priority_engine(ALGORITHM, target_bands)

priority_file = 'priorities.json'
rejected_file = 'invalid_cases.json'
try:
    os.remove(priority_file)
    os.remove(rejected_file)
except Exception as e:
    print(str(e))
finally:
    with open(priority_file, 'w+') as outfile:
        json.dump(calculated_priorities, outfile, indent=4)
        print(f"'{priority_file}' created")

    with open(rejected_file, 'w+') as outf:
        json.dump(invalid_cases, outf, indent=4)
        print(f"'{rejected_file}' created")

'priorities.json' created
'invalid_cases.json' created


In [22]:
user_data.loc[user_data['id'] == '15']

,id,location,class,op_freq,allowed_bands
14,15,loc25,ordinary,847,"[[824, 849]]"


In [23]:
user_data.loc[user_data['id'] == '235']

,id,location,class,op_freq,allowed_bands
234,235,loc21,government,2532,"[[2500, 2570], [3300, 3800], [1710, 1785]]"


In [24]:
user_data.loc[user_data['id'] == '54']

,id,location,class,op_freq,allowed_bands
53,54,loc30,emergency,2307,"[[2496, 2690], [2305, 2315]]"


In [25]:
user_data.loc[user_data['id'] == '206']

,id,location,class,op_freq,allowed_bands
205,206,loc8,scientific,2610,"[[2570, 2620], [1920, 1980]]"
